In [1]:
from dask.distributed import Client, LocalCluster

In [2]:
import importlib

In [3]:
cluster = LocalCluster()
client = Client(cluster)

/scratch/n74/hxr547/pyinstall/lib/python3.10/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34811 instead
  warnings.warn(
distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/90348801.gadi-pbs/dask-worker-space/worker-ts7qog6x', purging
distributed.diskutils - INFO - Found stale lock file and directory '/jobfs/90348801.gadi-pbs/dask-worker-space/worker-lell_jhr', purging


In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/34811/status,
Dashboard: /proxy/34811/status,Workers: 2
Total threads: 2,Total memory: 9.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43837,Workers: 2
Dashboard: /proxy/34811/status,Total threads: 2
Started: Just now,Total memory: 9.00 GiB
Comm: tcp://127.0.0.1:33603,Total threads: 1
Dashboard: /proxy/37371/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:35035,


In [5]:
print(client)

<Client: 'tcp://127.0.0.1:43837' processes=2 threads=2, memory=9.00 GiB>


In [6]:
pwd

'/home/547/hxr547'

In [7]:
cd /scratch/n74/hxr547/Projects/BNS_JT

/scratch/n74/hxr547/Projects/BNS_JT


In [8]:
from BNS_JT import config, variable, model, cpm, branch
from Trans import trans
import numpy as np

In [9]:
importlib.reload(branch)

<module 'BNS_JT.branch' from '/scratch/n74/hxr547/Projects/BNS_JT/BNS_JT/branch.py'>

In [10]:
pwd()

'/scratch/n74/hxr547/Projects/BNS_JT'

In [11]:
import time

In [12]:
cfg = config.Config('./Trans/tests/config_rbd.json')

In [13]:
# Arcs (components): P(X_i | GM = GM_ob ), i = 1 .. N (= nArc)
cpms = {}
varis = {}

# FIXME: only works for binary ATM
B = np.vstack([np.eye(cfg.no_ds), np.ones(cfg.no_ds)])

# only value is related to the scenario 
s1 = list(cfg.scenarios['scenarios'].keys())[0]
for k, values in cfg.scenarios['scenarios'][s1].items():
    varis[k] = variable.Variable(name=k, B=B, values=cfg.scenarios['damage_states'])
    cpms[k] = cpm.Cpm(variables = [varis[k]],
            no_child = 1,
            C = np.arange(len(values))[:, np.newaxis],
            p = values)


path_times = trans.get_all_paths_and_times(cfg.infra['ODs'].values(), cfg.infra['G'], key='time')




In [14]:
# FIXME: only works for binary ATM
lower = {k: 0 for k, _ in cfg.infra['edges'].items()}
upper = {k: 1 for k, _ in cfg.infra['edges'].items()}

variables = {k: varis[k] for k in cfg.infra['edges'].keys()}
for k, v in cfg.infra['ODs'].items():

    values = [np.inf] + sorted([y for _, y in path_times[v]], reverse=True)

    varis[k] = variable.Variable(name=k, B=np.eye(len(values)), values=values)

    tic = time.time()
    path_time_idx = trans.get_path_time_idx(path_times[v], varis[k])
    print(time.time() - tic)

    # FIXME
    tic = time.time()
    #pdb.set_trace()
    sb = branch.branch_and_bound_dask(path_time_idx, lower, upper, 1, client)
    print(time.time() - tic)


0.002166271209716797
/proxy/34811/status
b*: 1, sb: 1


distributed.worker - ERROR - Could not deserialize task
Traceback (most recent call last):
  File "/scratch/n74/hxr547/pyinstall/lib/python3.10/site-packages/distributed/worker.py", line 4428, in loads_function
    result = cache_loads[bytes_object]
  File "/scratch/n74/hxr547/pyinstall/lib/python3.10/site-packages/distributed/utils.py", line 1358, in __getitem__
    value = super().__getitem__(key)
  File "/apps/python3/3.10.4/lib/python3.10/collections/__init__.py", line 1106, in __getitem__
    raise KeyError(key)
KeyError: b'\x80\x04\x95\x1e\x00\x00\x00\x00\x00\x00\x00\x8c\rBNS_JT.branch\x94\x8c\x08fn_dummy\x94\x93\x94.'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/scratch/n74/hxr547/pyinstall/lib/python3.10/site-packages/distributed/worker.py", line 3471, in _maybe_deserialize_task
    function, args, kwargs = _deserialize(*ts.run_spec)
  File "/scratch/n74/hxr547/pyinstall/lib/python3.10/site-packages/distribute

ModuleNotFoundError: No module named 'BNS_JT'